In [ ]:
# Parameters - support for running the notebook from a shell script. Leave at None for interactive runs
date_param = None

In [ ]:
from datetime import datetime, timedelta
from datetime import datetime

# Reporting date range. Either parse the passed date parameter or use the value coded in the "else" case
start = datetime.strptime(date_param, "%Y-%m-%d") if date_param is not None else datetime(2025, 1, 1, 0, 0, 0)
end = start + timedelta(seconds=86399)

# Exclusion date ranges - this should be a list of tuples in which each tuple has two
# members, a start date and time and end date and time in that order
exclusions = []

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Set the Y-axis limits
y_min = 3
y_max = 15

# Smoothing parameter for LOWESS - this can be tuned in the range 0.03 to 0.06
# for CGM data with a 5-minute sampling interval
fraction = 0.05

# Define the rates of change (mmol/L/minute) and tolerance for identifying flat regions
flat_slope = 0.02
ultra_flat_slope = 0.01
curvature_tolerance = 0.01

# Set the time window for removing excursions with flat tops
# For a 5 minute sampling window, 24 * 5 minutes = 120 minutes
window_points = 21

# Define the local low quantile and the mmol/L tolerance above local floor
floor_quantile = 0.25
floor_tol=0.20

# To be included in a strict baseline, a point must be below this quantile for the day
global_quantile = None

# Excursion detection
excursion_slope=0.05
excursion_buffer_points=4

# Suffix for file exports (assumes only one analysis per day)
suffix = start.strftime("%Y-%m-%d")

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

## Data Load

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Remove any excluded ranges
df = remove_records_for_date_ranges(df, exclusions)

# Add hour of day
df["hour"] = df["date"].dt.hour

# Preview the data
df.head()

## Data Smoothing

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess

# Time axis in seconds from start
t = (df["date"] - df["date"].iloc[0]).dt.total_seconds().values
g = df["level"].astype(float).values

# LOWESS smoothing
smoothed = lowess(g, t, frac=fraction, return_sorted=False)
df["level_smooth"] = smoothed

# Preview the data
df.head()

## 1st and 2nd Derivative

In [ ]:
import numpy as np

# Slope in mmol/L per minute
dt_min = np.diff(t) / 60.0
dg = np.diff(df["level_smooth"].values)

# First derivative slope
slope = np.zeros_like(df["level_smooth"].values)
slope[1:] = dg / dt_min
df["slope"] = slope

# Simple curvature approximation (~second derivative)
curvature = np.zeros_like(df["level_smooth"].values)
curvature[2:] = np.diff(slope[1:])
df["curvature"] = curvature

# Preview the data
df.head()

## Flat Region Identification

In [ ]:
# Identify flat and "ultra" flat regions
df["is_flat"] = df["slope"].abs() < flat_slope
df["is_ultra_flat"] = (df["slope"].abs() < ultra_flat_slope) & (df["curvature"].abs() < curvature_tolerance)

# Preview the data
df.head()

## Baseline Region Identification

The next cell computes a strict baseline from CGM data by:

1. Taking only flat points ("is_flat" == True)
2. Keeping those near the local low in a rolling window
3. Requiring they are in the lower band of the day
4. Excluding anything inside/near high-slope excursions

In [ ]:
df = df.sort_values("date").reset_index(drop=True).copy()

s = df["level_smooth"].astype(float)

# --- 1. Local low envelope via rolling quantile ---
local_low = (
    s.rolling(
        window=window_points,
        center=True,
        min_periods=max(3, window_points // 4),
    )
    .quantile(floor_quantile)
)
df["baseline_local_low"] = local_low

# --- 2. Global cutoff for "low part of the day" ---
global_cutoff = s.quantile(global_quantile) if global_quantile is not None else None

# --- 3. Excursion zone: high slope + dilation ---
high_slope = df["slope"].abs() > excursion_slope

# Dilate high-slope mask with ±excursion_buffer_points neighbourhood
kernel = np.ones(2 * excursion_buffer_points + 1, dtype=int)
excursion_zone = np.convolve(high_slope.astype(int), kernel, mode="same") > 0

excursion_col = f"baseline_in_excursion_zone"
df[excursion_col] = excursion_zone

# --- 4. Final strict baseline mask ---
mask = df["is_flat"].astype(bool)
mask &= s <= (local_low + floor_tol)
mask &= ~excursion_zone

if global_cutoff is not None:
    mask &= s <= global_cutoff

df["baseline_is_true"] = mask
df["baseline_clean"] = s.where(mask)

# --- 5. Summary stats ---
baseline_series = df["baseline_clean"]

summary = {
    "date": start.strftime("%Y-%m-%d"),
    "mean": float(baseline_series.mean()),
    "std": float(baseline_series.std()),
    "n_points": int(mask.sum()),
    "global_cutoff": float(global_cutoff) if global_cutoff is not None else None,
    "window_points": int(window_points),
    "floor_quantile": float(floor_quantile),
    "floor_tol": float(floor_tol),
    "global_quantile": float(global_quantile) if global_quantile is not None else None,
    "excursion_slope": float(excursion_slope),
    "excursion_buffer_points": int(excursion_buffer_points),
}

# Overnight summary
overnight = df[(df["hour"] >= 0) & (df["hour"] <= 6)]
overnight_mean = overnight["baseline_clean"].mean()
summary["overnight_mean_0_6h"] = (
    float(overnight_mean) if not np.isnan(overnight_mean) else None
)

# Sanity check : Calculate how much of the data's included
summary["n_total"] = len(df)
summary["n_baseline"] = df["baseline_is_true"].sum()
summary["percent_baseline"] = 100 * summary["n_baseline"] / summary["n_total"]

df.head()

## Summary File Creation/Update

In [ ]:
from pathlib import Path

# Get the path to the summary file
export_folder = create_export_folder()
summary_csv = Path(export_folder) / "baseline_analysis.csv"
summary_exists = summary_csv.exists()

# Open the file for appending
with open(summary_csv.absolute(), "a", encoding="utf-8", newline="") as f:
    # If we're creating the file, add the headers
    if not summary_exists:
        headers = ",".join(summary.keys())
        f.write(headers + "\n")

    # Write the values from the summary structure
    values = ",".join(str(v) for v in summary.values())
    f.write(values + "\n")

## Data Export

In [ ]:
import pandas as pd

# Export the data to a spreadsheet
if export_spreadsheet:
    export_to_spreadsheet("glucose", f"glucose_baseline-{suffix}", {
        "Analysis": df,
        "Summary": pd.DataFrame(list(summary.items()), columns=["Property", "Value"])
    })

## Baseline Region Chart

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.plot(df["date"], df["level"], label="Raw", alpha=0.2)
plt.plot(df["date"], df["level_smooth"], label="Smoothed", alpha=0.5)
plt.plot(df["date"], df["baseline_clean"], label="Strict baseline", linewidth=2)

plt.legend()
plt.xlabel("Time")
plt.ylabel("Glucose (mmol/L)")
plt.title("CGM Strict Baseline with Plateau Peaks Removed")

# Export to PNG or PDF, if required
export_chart("glucose", "glucose_baseline", suffix, chart_export_format)

# Show the plot
plt.show()